In [1]:
!pip install --upgrade --quiet sagemaker>=2.215.0
!pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0 cloudpickle==2.2.1

In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.serve import SchemaBuilder
from sagemaker.serve import ModelBuilder
from sagemaker.serve.mode.function_pointers import Mode
import mlflow
from mlflow import MlflowClient
import boto3
import numpy as np
import pandas as pd
import os
from sklearn import datasets

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# SETTINGS

In [2]:
tracking_server_arn = "arn:aws:sagemaker:eu-west-1:211125740051:mlflow-tracking-server/Sample-server"
train_data_uri = "s3://sagemaker-bucket-ds/training-jobs/data/train/"
mlflow_experiment_name = "01-project-POL"
mlflow_register_name = "01-project-POL"
input_path = 's3://sagemaker-bucket-ds/training-jobs/data/inference_input/'
output_path = 's3://sagemaker-bucket-ds/training-jobs/data/inference_output/'

# SET MLFLOW

In [23]:
mlflow.set_tracking_uri(tracking_server_arn)
client = MlflowClient()
role = get_execution_role()
sagemaker_session = sagemaker.Session()

# GET THE MODEL PATH

In [4]:
registered_model = client.get_registered_model(name=mlflow_register_name)
run_id = registered_model.latest_versions[0].run_id
source_path = registered_model.latest_versions[0].source

In [5]:
sklearn_input = np.array([1.0, 2.0, 3.0, 4.0]).reshape(1, -1)
sklearn_output = 1
sklearn_schema_builder = SchemaBuilder(
    sample_input=sklearn_input,
    sample_output=sklearn_output,
)

In [10]:
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=sklearn_schema_builder,
    role_arn=role,
    model_metadata={"MLFLOW_MODEL_PATH": source_path},
)

In [12]:
built_model = model_builder.build()

ModelBuilder: INFO:     Downloading ML_FLOW/36/d73930aa982d4256bd908c0decdc3bb5/artifacts/model/MLmodel to /tmp/sagemaker/model-builder/c48dc69439eb11efa274767cd976fd19/MLmodel
ModelBuilder: INFO:     Downloading ML_FLOW/36/d73930aa982d4256bd908c0decdc3bb5/artifacts/model/conda.yaml to /tmp/sagemaker/model-builder/c48dc69439eb11efa274767cd976fd19/conda.yaml
ModelBuilder: INFO:     Downloading ML_FLOW/36/d73930aa982d4256bd908c0decdc3bb5/artifacts/model/metadata/MLmodel to /tmp/sagemaker/model-builder/c48dc69439eb11efa274767cd976fd19/metadata/MLmodel
ModelBuilder: INFO:     Downloading ML_FLOW/36/d73930aa982d4256bd908c0decdc3bb5/artifacts/model/metadata/conda.yaml to /tmp/sagemaker/model-builder/c48dc69439eb11efa274767cd976fd19/metadata/conda.yaml
ModelBuilder: INFO:     Downloading ML_FLOW/36/d73930aa982d4256bd908c0decdc3bb5/artifacts/model/metadata/python_env.yaml to /tmp/sagemaker/model-builder/c48dc69439eb11efa274767cd976fd19/metadata/python_env.yaml
ModelBuilder: INFO:     Downloadi

# CREATE ENDPOINT

In [18]:
predictor = built_model.deploy(initial_instance_count=1, instance_type="ml.m5.large")

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-07-04-10-09-20-607
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-07-04-10-09-21-235
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-07-04-10-09-21-235


-----------!

ModelBuilder: INFO:     Polling lineage artifact for model data in s3://sagemaker-eu-west-1-211125740051/sagemaker-scikit-learn-2024-07-04-09-59-35-605/serve.tar.gz
ModelBuilder: WARNING:     Lineage artifact for transformed model data is not auto-created within 120 seconds, skipping creation of lineage artifact for mlflow model path
ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


In [19]:
iris = datasets.load_iris()
predictor.predict(iris.data)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [24]:
sagemaker_session.delete_endpoint(endpoint_name=built_model.endpoint_name)

INFO:sagemaker:Deleting endpoint with name: sagemaker-scikit-learn-2024-07-04-10-09-21-235


In [29]:
!aws s3 rm s3://sagemaker-bucket-ds/training-jobs/data/inference_output/ --recursive

In [31]:
os.makedirs("03_training_code", exist_ok=True)

In [55]:
%%writefile 03_training_code/inference.py

import os
import pickle
import json
import pandas as pd
import io
from sagemaker_containers.beta.framework import worker

def model_fn(model_dir):
    with open(os.path.join(model_dir, "model.pkl"), "rb") as f:
        model = pickle.load(f)
    return model

def input_fn(input_data, content_type):
    if content_type == 'text/csv':
        return pd.read_csv(io.StringIO(input_data), header=None)
    else:
        raise ValueError(f"Unsupported content type: {content_type}")

def predict_fn(input_data, model):
    predictions = model.predict_proba(input_data)
    return predictions

def output_fn(prediction, content_type):
    if content_type == 'application/json':
        return worker.Response(json.dumps(prediction.tolist()), mimetype=content_type)
    elif content_type == 'text/csv':
        return worker.Response(pd.DataFrame(prediction).to_csv(index=False), mimetype=content_type)
    else:
        raise ValueError(f"Unsupported content type: {content_type}")

Overwriting 03_training_code/inference.py


In [45]:
!tar -czvf model.tar.gz model.pkl

model.pkl


In [46]:
# Define the new object key for the tar.gz file
tar_gz_object_key = 'ML_FLOW/36/d73930aa982d4256bd908c0decdc3bb5/artifacts/model/model.tar.gz'

# Upload the tar.gz file
s3.upload_file('model.tar.gz', bucket_name, tar_gz_object_key)

In [47]:
model_path = os.path.join(source_path, "model.tar.gz")
model_path

's3://sagemaker-bucket-ds/ML_FLOW/36/d73930aa982d4256bd908c0decdc3bb5/artifacts/model/model.tar.gz'

In [56]:
from sagemaker.sklearn.model import SKLearnModel

# Create an SKLearnModel object
sklearn_model = SKLearnModel(
    model_data=model_path,
    role=role,
    entry_point='inference.py',
    source_dir='03_training_code',
    framework_version='1.2-1'
)

In [ ]:
# Create the transformer object with a custom model name
transformer = sklearn_model.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=output_path,
    assemble_with='Line'
)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-07-04-10-55-35-350


In [ ]:
# Start the batch transform job
transformer.transform(
    data=input_path,
    content_type='text/csv',
    split_type='Line'
)